In [32]:
from bs4 import BeautifulSoup
import requests
from p00_general_functions import *
from dataclasses import dataclass
import datetime
import pandas as pd
from itertools import chain

@dataclass
class Event:
  key_: str
  series: str
  name: str
  venue: str
  city: str
  region: str
  country: str
  date: datetime
  tripleCrown: bool
  classes: list[str]

# *********************************
# ****** Scraping Functions *******
# *********************************

# Scrape Event Details in HTML from Racer X Vault:

def getEventHtml(s, year, type):

    # URL that scraper is looking up
    url = f'https://racerxonline.com/results/{year}/{type}/races'

    # Set result to url that is requested:
    result = s.get(url)
    doc = BeautifulSoup(result.text, "html.parser")

    return doc
    
def isTripleCrown(results):
  for result in results:
    if result.text.strip() == "Main Event 1":
      return True
    
  return False

def getClasses(classes):
  class_list = []
  for class_ in classes:
    if (class_.text == 'Articles' or class_.text ==''):
      pass
    else:
      class_list.append(class_.text)
  return class_list

def getEventDetails(html):
  races= html.find('ul', {'class': 'ui_list event_list'}).contents

  season_list = []
  for race in races:
    try:
      round_ = Event(
        series = race.select_one('div.tag').text,
        name = race.select_one('strong').text,
        venue = race.find('span',itemprop='name').text,
        city = race.find('span',itemprop='addressLocality').text,
        region = race.find('span',itemprop='addressRegion').text,
        country = race.find('span',itemprop='addressCountry').text,
        date = race.find('span',itemprop='startDate').text,
        key_ = genPrimaryKey(race.find('span',itemprop='startDate').text),
        tripleCrown = isTripleCrown(race.select('div.btn-group li')),
        classes = getClasses(race.select('div.btn-group a.btn'))
      )
      season_list.append(round_)
    except:
      pass
  return season_list

# *********************************
# ********* Main Function *********
# *********************************
s = requests.Session()
sx_events = []

print("This program will find Motocross and Supercross")
print("race events in the RacerX Vault.\n\n")

startYear = input("Enter Start Year: ")
endYear = input("Enter End Year: ")
raceType = input("Enter a race type (SX or MX): ")


for year in range(int(startYear),int(endYear)+1):
  sx_event = getEventDetails(getEventHtml(s,year,raceType.lower()))
  sx_events.append(sx_event)

This program will find Motocross and Supercross
race events in the RacerX Vault.




In [33]:

data = pd.DataFrame(list(chain.from_iterable(sx_events)))
data['date'] = pd.to_datetime(data['date'])
data.sort_values(by=['date'])


,key_,series,name,venue,city,region,country,date,tripleCrown,classes
16,20210116,Supercross,Houston 1 (East),NRG Stadium,Houston,TX,US,2021-01-16,False,"[450SX, 250SX East]"
15,20210119,Supercross,Houston 2 (East),NRG Stadium,Houston,TX,US,2021-01-19,False,"[450SX, 250SX East]"
14,20210123,Supercross,Houston 3 (East),NRG Stadium,Houston,TX,US,2021-01-23,False,"[450SX, 250SX East]"
13,20210130,Supercross,Indianapolis 1 (East),Lucas Oil Stadium,Indianapolis,IN,US,2021-01-30,False,"[450SX, 250SX East]"
12,20210202,Supercross,Indianapolis 2 (East),Lucas Oil Stadium,Indianapolis,IN,US,2021-02-02,False,"[450SX, 250SX East]"
11,20210206,Supercross,Indianapolis 3 (East),Lucas Oil Stadium,Indianapolis,IN,US,2021-02-06,False,"[450SX, 250SX East]"
10,20210213,Supercross,Orlando 1 (East),Camping World Stadium,Orlando,FL,US,2021-02-13,False,"[450SX, 250SX East]"
9,20210220,Supercross,Orlando 2 (West),Camping World Stadium,Orlando,FL,US,2021-02-20,False,"[450SX, 250SX West]"
8,20210306,Supercross,Daytona (West),Daytona International Speedway,Daytona,FL,US,2021-03-06,False,"[450SX, 250SX West]"
7,20210313,Supercross,Arlington 1 (West),AT&T Stadium,Arlington,TX,US,2021-03-13,False,"[450SX, 250SX West]"


<li><div class="tag">Supercross</div><div class="grid_event"><div class="event_main"><a class="event_title" href="/event/2022/salt-lake-city" title="In-Depth coverage of Salt Lake City - 2022 Supercross"><strong>Salt Lake City</strong></a> <span class="date" content="2022-05-07T20:00:00-04:00" data-format="F j, Y" itemprop="startDate">May 7, 2022</span> </div><div class="event_location"><a class="ui_link" href="/venue/rice-eccles-stadium"><span itemprop="location" itemscope="" itemtype="http://schema.org/Place" title="Supercross Race Results from Rice-Eccles Stadium - Salt Lake City, UT"><span class="link_title" itemprop="name">Rice-Eccles Stadium</span><br/><span itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress"><span itemprop="addressLocality">Salt Lake City</span>, <span itemprop="addressRegion">UT</span> <span itemprop="addressCountry">US</span></span> <img alt="United States" class="event_flag" src="//rx.iscdn.net/i/icons/flags/us.png"/></span></a></div></div><div class="btn-group"><a class="btn" href="/event/2022/salt-lake-city" title="In-Depth coverage of  Salt Lake City 2022 Supercross">Articles</a></div> <div class="btn-group"><a class="btn" href="/results/2022/salt-lake-city/450sx" title="Salt Lake City 2022 - 450SX Results">450SX</a> <ul class="dropdown_nav_content"><li><a href="/results/2022/salt-lake-city/450sx" title="Salt Lake City 2022 - 450SX Results">Overall Results</a></li> <li><a href="/results/2022/salt-lake-city/450sx/main-event" title="Salt Lake City 2022 - 450SX Main Event Results">Main Event</a></li><li><a href="/results/2022/salt-lake-city/450sx/lcq" title="Salt Lake City 2022 - 450SX LCQ Results">LCQ</a></li><li><a href="/results/2022/salt-lake-city/450sx/heat-2" title="Salt Lake City 2022 - 450SX Heat 2 Results">Heat 2</a></li><li><a href="/results/2022/salt-lake-city/450sx/heat-1" title="Salt Lake City 2022 - 450SX Heat 1 Results">Heat 1</a></li><li><a href="/results/2022/salt-lake-city/450sx/combined-qualifying" title="Salt Lake City 2022 - 450SX Combined Qualifying Results">Combined Qualifying</a></li><li><a href="/results/2022/salt-lake-city/450sx/group-c-qualifying-2" title="Salt Lake City 2022 - 450SX Group C Qualifying 2 Results">Group C Qualifying 2</a></li><li><a href="/results/2022/salt-lake-city/450sx/group-b-qualifying-2" title="Salt Lake City 2022 - 450SX Group B Qualifying 2 Results">Group B Qualifying 2</a></li><li><a href="/results/2022/salt-lake-city/450sx/group-a-qualifying-2" title="Salt Lake City 2022 - 450SX Group A Qualifying 2 Results">Group A Qualifying 2</a></li><li><a href="/results/2022/salt-lake-city/450sx/group-c-qualifying-1" title="Salt Lake City 2022 - 450SX Group C Qualifying 1 Results">Group C Qualifying 1</a></li><li><a href="/results/2022/salt-lake-city/450sx/group-b-qualifying-1" title="Salt Lake City 2022 - 450SX Group B Qualifying 1 Results">Group B Qualifying 1</a></li><li><a href="/results/2022/salt-lake-city/450sx/group-a-qualifying-1" title="Salt Lake City 2022 - 450SX Group A Qualifying 1 Results">Group A Qualifying 1</a></li> </ul><a class="btn dropdown_nav_btn" href="#"><i class="i_arrow-down"></i></a> </div><div class="btn-group"><a class="btn" href="/results/2022/salt-lake-city/250sx-showdown" title="Salt Lake City 2022 - 250SX Showdown Results">250SX Showdown</a> <ul class="dropdown_nav_content"><li><a href="/results/2022/salt-lake-city/250sx-showdown" title="Salt Lake City 2022 - 250SX Showdown Results">Overall Results</a></li> <li><a href="/results/2022/salt-lake-city/250sx-showdown/main-event" title="Salt Lake City 2022 - 250SX Showdown Main Event Results">Main Event</a></li><li><a href="/results/2022/salt-lake-city/250sx-showdown/lcq" title="Salt Lake City 2022 - 250SX Showdown LCQ Results">LCQ</a></li><li><a href="/results/2022/salt-lake-city/250sx-showdown/east-heat" title="Salt Lake City 2022 - 250SX Showdown East Heat Results">East Heat</a></li><li><a href="/results/2022/salt-lake-city/250sx-showdown/west-heat" title="Salt Lake City 2022 - 250SX Showdown West Heat Results">West Heat</a></li><li><a href="/results/2022/salt-lake-city/250sx-showdown/combined-qualifying" title="Salt Lake City 2022 - 250SX Showdown Combined Qualifying Results">Combined Qualifying</a></li><li><a href="/results/2022/salt-lake-city/250sx-showdown/east-qualifying-2" title="Salt Lake City 2022 - 250SX Showdown East Qualifying 2 Results">East Qualifying 2</a></li><li><a href="/results/2022/salt-lake-city/250sx-showdown/west-qualifying-2" title="Salt Lake City 2022 - 250SX Showdown West Qualifying 2 Results">West Qualifying 2</a></li><li><a href="/results/2022/salt-lake-city/250sx-showdown/overflow-qualifying-2" title="Salt Lake City 2022 - 250SX Showdown Overflow Qualifying 2 Results">Overflow Qualifying 2</a></li><li><a href="/results/2022/salt-lake-city/250sx-showdown/west-qualifying-1" title="Salt Lake City 2022 - 250SX Showdown West Qualifying 1 Results">West Qualifying 1</a></li><li><a href="/results/2022/salt-lake-city/250sx-showdown/east-qualifying-1" title="Salt Lake City 2022 - 250SX Showdown East Qualifying 1 Results">East Qualifying 1</a></li><li><a href="/results/2022/salt-lake-city/250sx-showdown/overflow-qualifying-1" title="Salt Lake City 2022 - 250SX Showdown Overflow Qualifying 1 Results">Overflow Qualifying 1</a></li> </ul><a class="btn dropdown_nav_btn" href="#"><i class="i_arrow-down"></i></a> </div><div class="btn-group"><a class="btn" href="/results/2022/salt-lake-city/250sx-futures" title="Salt Lake City 2022 - 250SX Futures Results">250SX Futures</a> <ul class="dropdown_nav_content"><li><a href="/results/2022/salt-lake-city/250sx-futures" title="Salt Lake City 2022 - 250SX Futures Results">Overall Results</a></li> <li><a href="/results/2022/salt-lake-city/250sx-futures/main-event" title="Salt Lake City 2022 - 250SX Futures Main Event Results">Main Event</a></li><li><a href="/results/2022/salt-lake-city/250sx-futures/combined-qualifying" title="Salt Lake City 2022 - 250SX Futures Combined Qualifying Results">Combined Qualifying</a></li><li><a href="/results/2022/salt-lake-city/250sx-futures/combined-qualifying" title="Salt Lake City 2022 - 250SX Futures Combined Qualifying Results">Combined Qualifying</a></li> </ul><a class="btn dropdown_nav_btn" href="#"><i class="i_arrow-down"></i></a> </div> </li>

<li><div class="tag">Supercross</div><div class="grid_event"><div class="event_main"><a class="event_title" href="/event/2022/st-louis" title="In-Depth coverage of St. Louis - 2022 Supercross"><strong>St. Louis</strong></a> <span class="date" content="2022-04-09T20:00:00-04:00" data-format="F j, Y" itemprop="startDate">April 9, 2022</span> </div><div class="event_location"><a class="ui_link" href="/venue/dome-at-americas-center"><span itemprop="location" itemscope="" itemtype="http://schema.org/Place" title="Supercross Race Results from Dome at America's Center - St. Louis, MO"><span class="link_title" itemprop="name">Dome at America's Center</span><br/><span itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress"><span itemprop="addressLocality">St. Louis</span>, <span itemprop="addressRegion">MO</span> <span itemprop="addressCountry">US</span></span> <img alt="United States" class="event_flag" src="//rx.iscdn.net/i/icons/flags/us.png"/></span></a></div></div><div class="btn-group"><a class="btn" href="/event/2022/st-louis" title="In-Depth coverage of  St. Louis 2022 Supercross">Articles</a></div> <div class="btn-group"><a class="btn" href="/results/2022/st-louis/450sx" title="St. Louis 2022 - 450SX Results">450SX</a> <ul class="dropdown_nav_content"><li><a href="/results/2022/st-louis/450sx" title="St. Louis 2022 - 450SX Results">Overall Results</a></li> <li><a href="/results/2022/st-louis/450sx/main-event-3" title="St. Louis 2022 - 450SX Main Event 3 Results">Main Event 3</a></li><li><a href="/results/2022/st-louis/450sx/main-event-2" title="St. Louis 2022 - 450SX Main Event 2 Results">Main Event 2</a></li><li><a href="/results/2022/st-louis/450sx/main-event-1" title="St. Louis 2022 - 450SX Main Event 1 Results">Main Event 1</a></li><li><a href="/results/2022/st-louis/450sx/lcq" title="St. Louis 2022 - 450SX LCQ Results">LCQ</a></li><li><a href="/results/2022/st-louis/450sx/combined-qualifying" title="St. Louis 2022 - 450SX Combined Qualifying Results">Combined Qualifying</a></li><li><a href="/results/2022/st-louis/450sx/group-c-qualifying-2" title="St. Louis 2022 - 450SX Group C Qualifying 2 Results">Group C Qualifying 2</a></li><li><a href="/results/2022/st-louis/450sx/group-b-qualifying-2" title="St. Louis 2022 - 450SX Group B Qualifying 2 Results">Group B Qualifying 2</a></li><li><a href="/results/2022/st-louis/450sx/group-a-qualifying-2" title="St. Louis 2022 - 450SX Group A Qualifying 2 Results">Group A Qualifying 2</a></li><li><a href="/results/2022/st-louis/450sx/group-c-qualifying-1" title="St. Louis 2022 - 450SX Group C Qualifying 1 Results">Group C Qualifying 1</a></li><li><a href="/results/2022/st-louis/450sx/group-b-qualifying-1" title="St. Louis 2022 - 450SX Group B Qualifying 1 Results">Group B Qualifying 1</a></li><li><a href="/results/2022/st-louis/450sx/group-a-qualifying-1" title="St. Louis 2022 - 450SX Group A Qualifying 1 Results">Group A Qualifying 1</a></li> </ul><a class="btn dropdown_nav_btn" href="#"><i class="i_arrow-down"></i></a> </div><div class="btn-group"><a class="btn" href="/results/2022/st-louis/250sx-east" title="St. Louis 2022 - 250SX East Results">250SX East</a> <ul class="dropdown_nav_content"><li><a href="/results/2022/st-louis/250sx-east" title="St. Louis 2022 - 250SX East Results">Overall Results</a></li> <li><a href="/results/2022/st-louis/250sx-east/main-event-3" title="St. Louis 2022 - 250SX East Main Event 3 Results">Main Event 3</a></li><li><a href="/results/2022/st-louis/250sx-east/main-event-2" title="St. Louis 2022 - 250SX East Main Event 2 Results">Main Event 2</a></li><li><a href="/results/2022/st-louis/250sx-east/main-event-1" title="St. Louis 2022 - 250SX East Main Event 1 Results">Main Event 1</a></li><li><a href="/results/2022/st-louis/250sx-east/lcq" title="St. Louis 2022 - 250SX East LCQ Results">LCQ</a></li><li><a href="/results/2022/st-louis/250sx-east/combined-qualifying" title="St. Louis 2022 - 250SX East Combined Qualifying Results">Combined Qualifying</a></li><li><a href="/results/2022/st-louis/250sx-east/group-a-qualifying-2" title="St. Louis 2022 - 250SX East Group A Qualifying 2 Results">Group A Qualifying 2</a></li><li><a href="/results/2022/st-louis/250sx-east/group-b-qualifying-2" title="St. Louis 2022 - 250SX East Group B Qualifying 2 Results">Group B Qualifying 2</a></li><li><a href="/results/2022/st-louis/250sx-east/group-c-qualifying-2" title="St. Louis 2022 - 250SX East Group C Qualifying 2 Results">Group C Qualifying 2</a></li><li><a href="/results/2022/st-louis/250sx-east/group-a-qualifying-1" title="St. Louis 2022 - 250SX East Group A Qualifying 1 Results">Group A Qualifying 1</a></li><li><a href="/results/2022/st-louis/250sx-east/group-b-qualifying-1" title="St. Louis 2022 - 250SX East Group B Qualifying 1 Results">Group B Qualifying 1</a></li><li><a href="/results/2022/st-louis/250sx-east/group-c-qualifying-1" title="St. Louis 2022 - 250SX East Group C Qualifying 1 Results">Group C Qualifying 1</a></li> </ul><a class="btn dropdown_nav_btn" href="#"><i class="i_arrow-down"></i></a> </div> </li>